In [1]:
import urllib.request
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
import collections
import json


### Me interesa calcular las coordenadas de los pisos a la mejor ubicacion de Valladolid y de Bilbao

In [2]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="My_First_Project")
loc_centro_vall=geolocator.geocode("Plaza Mayor, Valladolid" )
loc_centro_bilb=geolocator.geocode('Casco Viejo, Bilbao')


Urls 

In [3]:
html_vall=('https://www.pisos.com/venta/pisos-valladolid_capital/')

In [4]:
html_bilb=('https://www.pisos.com/venta/pisos-bilbao/')

In [5]:
def last_page(selection):
    last_page=[]
    url=(selection)
    r=requests.get(url)
    res=BeautifulSoup(r.text,"lxml")
    page=res.findAll('div', {"class": 'pager'})
    last_page=[]
    for i in page:
        pages=i.findAll('a', {"class": 'item'})
        last_page.append(pages[-1].text)
    
    return last_page

In [6]:
last_page_bilb=last_page(html_bilb)

In [7]:
last_page_vall=last_page(html_vall)

In [8]:
def pager(html,last_page):
    pages=[]
    for page in range(1,int(last_page[0])):
        try:
            url=(html+'{}/'.format(page))
        except HTTPERROR as e:
    
            print(e)

        except URLError:

            print("Server down or incorrect domain")
    
        else:
            r=requests.get(url)
            soup=BeautifulSoup(r.text,"lxml")
        
            if soup.title is None:

                print("Tag not found")
            pages.append(soup)
    return pages

In [9]:
pages_vall=pager(html_vall,last_page_vall)

In [10]:
pages_bilb=pager(html_bilb,last_page_bilb)

In [11]:
pages=pages_vall+pages_bilb

### Extraigo el contenedor (cada anuncio) y el link asociado para extraer las caracteristicas

In [12]:
def container(pages):
   
    containers=[]

    for i in range (0,len(pages)):

        containers.append(pages[i].findAll('div', {"itemtype": "http://schema.org/SingleFamilyResidence"}))
        
    return (containers)
   


In [13]:
def links(pages):
   
    links=[]

    for i in range (0,len(pages)):

        links.append(pages[i].findAll('div', {"class": "gridList Listado"}))
        
    return (links)



In [14]:
container_vall=container(pages_vall)
container_bilb=container(pages_bilb)


links_vall=links(pages_vall)
links_bilb=links(pages_bilb)


CONTAINERS

In [15]:
def names(container):

    names=[]


    for container in container:
        for i in container:


            name=i.find('meta', {'itemprop': 'name'})
            names.append(name)

    return(names)      
   

In [16]:
names_vall=names(container_vall)
names_bilb=names(container_bilb)

In [17]:
len(names_vall)

2460

### Saco el nombre de la calle de cada vivienda y el título de cada anuncio para poder sacar las coordenadas de las viviendas con la librería Geopy

In [18]:
def sacar_titulo(names):

    names_string=[]
    for l in names:
        names_string.append(str(l))

    titulos=[]
    for l in names_string:
        l1=l.replace('<meta content="','')
        l2=l1.replace('" itemprop="name"/>','')    
        titulos.append(l2.strip())
    

    return (titulos)


In [19]:
titulos_vall=sacar_titulo(names_vall)
titulos_bilb=sacar_titulo(names_bilb)


In [20]:
def calles(titulos):
    calles=[]
    for i in titulos:
        en=i.find('en ')
        punto=i.find('.')
        guion=i.find('-')
        coma=i.find(',')
        if punto == -1 :
            if guion !=-1 & coma !=-1:        
                if (coma <= guion):
                    men=i[en+3:coma]
            else:
                men=i[en+3:guion]
            if guion !=-1 & coma ==-1:
                men=i[en+3:guion]
        else: 
            men=i[en+3:coma]
        if guion ==-1:
            if punto !=-1 & coma !=-1:        
                if (coma <= punto):
                    men=i[en+3:coma]
            else:
                cmen=i[en+3:punto]
            if punto !=-1 & coma ==-1:
                men=i[en+3:punto]
            else: 
                men=i[en+3:coma]
        if coma ==-1:
            if punto !=-1 & guion !=-1:        
                if (guion <= punto):
                    men=i[en+3:guion]
                else:
                    men=i[en+3:punto]
            if punto !=-1 & guion ==-1:
                men=i[en+3:punto]
            else: 
                men=i[en+3:guion]
        if punto==-1 & coma==-1 & guion==-1:
            men=i[en+3:]
        if punto!=-1 & coma!=-1 & guion!=-1:
            if punto<=guion & punto<=coma:
                men=i[en+3:punto]    
            if guion<=punto & guion<=coma:
                men=i[en+3:guion]
            if coma<=punto & coma<=guion:
                men=i[en+3:coma]

        calles.append(men)
    return (calles)

In [22]:
calles_vall=calles(titulos_vall)
calles_bilb=calles(titulos_bilb)


In [23]:
loc_lat=[]
loc_lon=[]
for calle in calles_vall:
    try:
        location = geolocator.geocode("%s, Valladolid" %calle)
        loc_lat.append(location.latitude)
        loc_lon.append(location.longitude)
    except AttributeError:
        loc_lat.append('null')
        loc_lon.append('null')


KeyboardInterrupt: 

In [89]:
loc_lat_bilb=[]
loc_lon_bilb=[]
for calle in calles_bilb:
    try:
        location = geolocator.geocode("%s, Bilbao" %calle)
        loc_lat_bilb.append(location.latitude)
        loc_lon_bilb.append(location.longitude)
    except AttributeError:
        loc_lat_bilb.append('null')
        loc_lon_bilb.append('null')


### Una vez tengo las long y lat de las viviendas de Valladolid y Bilbao, calculo la distancia al centro

In [24]:
def calcular_dist_centro(loc_lat,loc_lon,lat,lon):
    coord=[]
    dist_centro=[]
    coord_centro=(lat,lon)
    from geopy.distance import geodesic

    for i in range(0,len(loc_lat)):
        coorde=(loc_lat[i], loc_lon[i])
        coord.append(coorde)

    for i in range(0, len(coord)):
        if coord[i]!=('null','null'):
            dist_centro.append(geodesic(coord[i],coord_centro).kilometers)
        else:
            dist_centro.append('null')
    return (dist_centro)

In [25]:
dist_centro_vall=calcular_dist_centro(loc_lat, loc_lon, loc_centro_vall.latitude, loc_centro_vall.longitude)
dist_centro_bilb=calcular_dist_centro(loc_lat_bilb, loc_lon_bilb, loc_centro_bilb.latitude, loc_centro_bilb.longitude)

NameError: name 'loc_lat_bilb' is not defined

In [157]:
len(dist_centro_vall)
len(dist_centro_bilb)

1980

In [387]:
location_lon=loc_lon+loc_lon_bilb
location_lat=loc_lat+loc_lat_bilb
dist_centro=dist_centro_vall + dist_centro_bilb

### Saco el resto de características comunes para Vall y Bilbao

In [26]:
containers2=container_vall+container_bilb

In [27]:

    prices=[]
    locations=[]
    pictures=[]
    charact=[]
    meters=[]
    latitudes=[]
    longitudes=[]
    names=[]
    urls=[]

    for container in containers2:
        for i in container:
            price=i.find('div', {"class": "priceBox"}).text 
            prices.append(price)

            location=i.find('div', {"class": "location"}).text 
            locations.append(location)

            picture=i.find('div', {'class':'overInfo'})
            pictures.append(picture)


            char=i.find('div', {"class": "characteristics"})
            charact.append(char)

            meter=i.find('div', {"item"}).text
            meters.append(meter)    

            latitude=i.find('meta', {'itemprop': 'latitude'})
            latitudes.append(latitude)

            longitude=i.find('meta', {'itemprop': 'longitude'})
            longitudes.append(longitude)

            name=i.find('meta', {'itemprop': 'name'})
            names.append(name)

            url=i.find('meta', {'itemprop': 'url'})
            urls.append(url)
    
   

### Saco las coordenadas que me aporta la página 

In [28]:
longitudes_string=[]
for l in longitudes:
    longitudes_string.append(str(l))

long=[]
for l in longitudes_string:
    l1=l.replace('<meta content="','')
    l2=l1.replace('" itemprop="longitude"/>','')    
    long.append(l2.strip())

latitudes_string=[]
for l in latitudes:
    latitudes_string.append(str(l))

lat=[]
for l in latitudes_string:
    l1=l.replace('<meta content="','')
    l2=l1.replace('" itemprop="latitude"/>','')    
    lat.append(l2.strip())

In [29]:
lat
len(lat)

4380

## URLS

In [30]:
urls_string=[]
for l in urls:
    urls_string.append(str(l))

In [31]:
urls_replace=[]
for l in urls_string:
    l1=l.replace('<meta content="','')
    l2=l1.replace('" itemprop="url"/>','')
    urls_replace.append(l2.strip())

In [32]:
len(urls_replace)

4380

In [34]:
urls_list=[]
for i in range(0,len(urls_replace)):
    urls_list.append('https://www.pisos.com'+urls_replace[i])

In [43]:
urls_list[4000]

'https://www.pisos.com/comprar/piso-iralabarri48012-93334034824_100500/'

In [36]:
len(urls_list)

4380

CARACTERISTICAS EXTRAS

In [41]:
def caracteristicas_extras (urls_list):
    caract_extras=[]
    for i in range(0,len(urls_list)): 

        try:
            url=urls_list[i]
        except HTTPERROR as e:

            print(e)

        except URLError:

            print("Server down or incorrect domain")

        else:
            r=requests.get(url)
            soup=BeautifulSoup(r.text,"lxml")

            if soup.title is None:

                print("Tag not found")
            caract_extras.append(soup)
            
    return (caract_extras)



In [ ]:
caract_extras1=caracteristicas_extras(urls_list)

DESCRIPCION

In [ ]:
description=[] #FEAT
for char in caract_extras1:
    description.append(char.find('div', {"id": "descriptionBody"}))
    

In [300]:
descripcion_string=[]
for char in description:
    descripcion_string.append(str(char))

In [307]:
    descripcion=[]
    for l in descripcion_string:
        l1=l.replace('<div class="description-container description-body" id="descriptionBody">\r\n','')
        l2=l1.replace('</div>','')    
        descripcion.append(l2.strip()) 



TITULO

In [195]:
titulo=[]
for char in caract_extras1:
     titulo.append(char.title.text)

In [196]:
titulo[0]

'Piso en venta en Calle de Concha Velasco en Paseo Zorrilla-Campo Grande-Cuatro de Marzo por 114.900 €'

TIPO_HOME

In [263]:
title_split=[]
for char in titulo:
    title_split.append(char.split(' '))
    

In [264]:
tipo_home=[]
for char in title_split:
    tipo_home.append(char[0])
    
    

In [265]:
vocabulary_home=collections.Counter(tipo_home)

In [266]:
vocab_home=list(vocabulary_home) #FEAT

In [267]:
vocab_home

['Piso',
 'Apartamento',
 'Casa',
 'Ático',
 'Chalet',
 'Dúplex',
 'Estudio',
 'Pisos',
 'Finca',
 'Loft']

#### PARA SACAR LAS VARIABLES ascensor, garaje, trastero

#### CARACT_EXTRAS_

In [40]:
caract_extras_=[]
for char in caract_extras1:
    caract_extras_.append(char.findAll('li', {"class": "charblock-element element-with-bullet"}))

NameError: name 'caract_extras1' is not defined

In [280]:
caract_extras_[0]
description[2]

<div class="description-container description-body" id="descriptionBody">
            Vivienda a reformar, en pleno centro de valladolid junto a plaza zorrilla. Semisotano. Cuatro estancias. Baño con bañera y termo. Sin cocina. Interior, orientado hacia patio de manzana. Radiadores electricos. Suelo tarima. Oportunidad. Producto bancario
        </div>

In [270]:
def garaje_trastero(sub,reference_list):
    list=[]
    count=0
    for char in reference_list:
        for char2 in char:
            if sub in (char2.span.text).lower():
                countsi=1
                count+=countsi

            else:
                count=count
        if count>0:

            list.append(1)
        else:
            list.append(0)
        count=0   
    
    return (list)


In [271]:
Garaje=garaje_trastero('garaje',caract_extras_)

In [272]:
Trastero=garaje_trastero('trastero',caract_extras_)

In [312]:
Ascensor=[]
count=0
for char in caract_extras_:
    for char2 in char:
        if 'ascensor' in (char2.span.text).lower():
            countsi=1
            count+=countsi
           
        else:
            count+=count
    if count>0:
            
        Ascensor.append(1)
    else:
        Ascensor.append(0)
    count=0
    
    
    

GARAJE_DESC, veo si garaje está en descripcion del anuncio, ya que no está indicado como característica siempre

In [313]:
garaje_desc=[] #feat
for i in range(0,len(descripcion)):
      
    if 'garaje' in descripcion[i].lower():
        garaje_desc.append(1)
    else:
        garaje_desc.append(0)
        

In [314]:
len(Garaje)

4440

ASCENSOR_DESC, veo si ascensor está en descripción del anuncio, ya que no está indicado como característica siempre

In [315]:
ascensor_desc=[] #feat

for i in range(0,len(descripcion)):
      
    #if 'piso' in titulo[i].lower() or 'ático' in titulo[i].lower()  or 'dúplex' in titulo[i].lower() or 'apartamento' in titulo[i].lower() or 'estudio' in titulo[i].lower():
    if 'ascensor' in descripcion[i].lower():
        ascensor_desc.append(1)
    else:
        ascensor_desc.append(0) 
  
    #elif 'chalet' in titulo[i].lower() or 'casa' in titulo[i].lower() or 'finca' in titulo[i].lower():
        #ascensor_desc.append('na')
   
    #else:
        #ascensor_desc.append('No info')    
      
        
        
         

TRASTERO_DESC

In [316]:
trastero_desc=[] #FEAT
for i in range(0,len(descripcion)):
      
    if 'trastero' in descripcion[i].lower():
        trastero_desc.append(1)
    else:
        trastero_desc.append(0)
        

PARQUET_DESC

In [317]:
parquet_desc=[] #FEAT
for i in range(0,len(descripcion)):
      
    if 'parquet' in descripcion[i].lower():
        parquet_desc.append(1)
    else:
        parquet_desc.append(0)
        

JARDIN_DESC

In [318]:
jardin_desc=[] #FEAT
for i in range(0,len(descripcion)):
      
    if 'jardin' in descripcion[i].lower():
        jardin_desc.append(1)
    else:
        jardin_desc.append(0)
        

#### PARA SACAR LAS VARIABLES baños, plantas, habitaciones

#### CARACT_EXTRAS_2

In [39]:
caract_extras_2=[]
for char in caract_extras1:
    caract_extras_2.append(char.findAll('ul', {"class": "charblock-list"}))

NameError: name 'caract_extras1' is not defined

In [322]:
caract_extras_2_string=[]
for char in caract_extras_2:
        caract_extras_2_string.append(str(char))

In [323]:
caract_extras_2_split=[]
for char in caract_extras_2_string:
    caract_extras_2_split.append(char.split('\n'))


In [324]:
def baños_hab_planta(sub1, sub2,reference_list):
    list=[]
    count=0
    for i in range(0,len(reference_list)):
        for j in range(0,len(reference_list[i])):
            if sub1 in reference_list[i][j] or sub2 in reference_list[i][j]:
                countsi=1
                count+=countsi
                z=j
            else:
                count+=count

        if count>1:

            list.append(reference_list[i][z+1])
        else:
            list.append('No info')
        count=0
            
    return list            
        

In [325]:
Baños=baños_hab_planta('banos','Baños',caract_extras_2_split)

In [326]:
Plantas=baños_hab_planta('planta','Planta',caract_extras_2_split)

In [327]:
Habitaciones=baños_hab_planta('habitaciones','Habitaciones',caract_extras_2_split)

BAÑOS

In [328]:
baños=[] #feat
for char in Baños:
    char1=char.replace("<span>:",'')
    char2=char1.replace('</span>','')
    
    baños.append(char2)
        

PLANTA

In [329]:
planta=[] #feat
for char in Plantas:
    char1=char.replace("<span>:",'')
    char2=char1.replace('ª','')
    char3=char2.replace('</span>','')
    
    planta.append(char3)

In [332]:
def unique_words (list):
    keys=set(list)
    return keys    

In [333]:
vocabulary_planta=collections.Counter(planta)

In [334]:
keys_planta=list(vocabulary_planta)


HABITACIONES

In [330]:
habitaciones=[] #feat
for char in Habitaciones:
    char1=char.replace("<span>:",'')
    char2=char1.replace('</span>','')

    habitaciones.append(char2)

In [331]:
len(habitaciones)

4440

OBRA_NUEVA

In [344]:
extras_obra=[]
for char in charact: # charact lo habia sacado al principio de containers
    ch=char.find('div', {"item extra"})  
    extras_obra.append(ch)   

In [345]:
extras_obra_2=[]
for extra in extras_obra:
    try:
        extras_obra_2.append(extra.text)
    except AttributeError:
        extras_obra_2.append('0')

In [346]:
obra_nueva=[]
for extra in extras_obra_2:
    if 'Obra nueva' in extra:
        obra_nueva.append(1)
    else:
        obra_nueva.append(0)

METROS

In [358]:
metros=[]
for meter in meters:
    meter1=meter.replace('m²','')
    metros.append(meter1.strip())

In [359]:
len(metros) #feat

4440

PRECIOS

In [360]:
precios_strip=[]
for price in prices:
    precios_strip.append(price.strip())
    

In [361]:
precios_split=[]
for price in precios_strip:
    precios_split.append(price.split('€'))

In [362]:
precios=[]
for i in range(0,len(precios_split)):
    precios.append(precios_split[i][0])

In [363]:
len(precios) #FEAT

4440

BARRIOS

In [364]:
barrios=[]
for location in locations:
    barrios.append(location.strip())  
    

In [365]:
len(barrios) #FEAT

4440

FUNCION PARA SACAR EL NOMBRE DE LOS BARRIOS

In [366]:
def barrios_ciudad (sub, reference_list):
    list=[]
    for b in reference_list:
        if sub in b:
            list.append(b)
    return list
    

In [367]:
barrios_vall=barrios_ciudad('Valladolid',barrios)


In [368]:
barrios_vall

['Paseo Zorrilla-Campo Grande-Cuatro de Marzo (Valladolid Capital)',
 'Paseo Zorrilla-Campo Grande-Cuatro de Marzo (Valladolid Capital)',
 'Centro (Valladolid Capital)',
 'Las Delicias (Valladolid Capital)',
 'Las Villas-Covaresa-Parque Alameda-La Rubia (Valladolid Capital)',
 'Centro (Valladolid Capital)',
 'Las Villas-Covaresa-Parque Alameda-La Rubia (Valladolid Capital)',
 'Paseo Zorrilla-Campo Grande-Cuatro de Marzo (Valladolid Capital)',
 'Centro (Valladolid Capital)',
 'Centro (Valladolid Capital)',
 'Las Delicias (Valladolid Capital)',
 'Parquesol (Valladolid Capital)',
 'Centro (Valladolid Capital)',
 'Parquesol (Valladolid Capital)',
 'Las Villas-Covaresa-Parque Alameda-La Rubia (Valladolid Capital)',
 'Centro (Valladolid Capital)',
 'Parquesol (Valladolid Capital)',
 'Parquesol (Valladolid Capital)',
 'Paseo Zorrilla-Campo Grande-Cuatro de Marzo (Valladolid Capital)',
 'Las Villas-Covaresa-Parque Alameda-La Rubia (Valladolid Capital)',
 'Las Villas-Covaresa-Parque Alameda-La 

CIUDAD

In [369]:
ciudad=[] #FEAT
for i in range(0,len(barrios)):
    if 'Valladolid' in barrios[i]:
        ciudad.append('Valladolid')
    
    elif 'Bilbao' in barrios[i]:
        ciudad.append('Bilbao')
        
    
   
        
        

RENTA DISPONIBLE MEDIA POR BARRIO

In [370]:
renta_barrios_bilb={
'San Frantzisko (Distrito Ibaiondo. Bilbao)':13668,
'Solokoetxe (Distrito Ibaiondo. Bilbao)':18304,
'Santutxu (Distrito Begoña-Santutxu. Bilbao)':16822,
'Begoña-Santutxu (Bilbao)':22492,
'Zurbaran-Arabella (Distrito Uribarri. Bilbao)':18625,
'Iturrigorri-Peñascal (Distrito Errekalde. Bilbao)':10193,
'Atxuri (Distrito Ibaiondo. Bilbao)':16434,
'Castaños (Distrito Uribarri. Bilbao)':29160,
'San Adrian (Distrito Ibaiondo. Bilbao)':17704,
'Abando Hurtado de Amézaga (Distrito Abando. Bilbao)':35944,
'Arangoiti (Distrito Deustu. Bilbao)':14718,
'Indautxu (Bilbao)':35702,
'Iturrialde (Distrito Ibaiondo. Bilbao)':19404,
'Uribarri (Distrito Uribarri. Bilbao)':16549,
'Iralabarri (Distrito Errekalde. Bilbao)':18667,
'Altamira (Distrito Basurtu-Zorrotza. Bilbao)':13853,
'Abando (Bilbao)': 35944,
'Doneperiaga-Deustuibarra San Pedro-La Ribera (Distrito Deustu. Bilbao)':23759,
'Bolueta (Distrito Begoña-Santutxu. Bilbao)':14943,
'Abandoibarra-Guggenheim (Distrito Abando. Bilbao)': 35207,
'Masustegi-Mintegitxueta (Distrito Basurtu-Zorrotza. Bilbao)':18605,
'Ibarrekolanda (Distrito Deustu. Bilbao)':21113,
'Miribilla (Distrito Ibaiondo. Bilbao)':26165,
'Errekaldeberri-Larraskitu (Distrito Errekalde. Bilbao)':16293,
'Uretamendi (Distrito Errekalde. Bilbao)':11860,
'Otxarkoaga (Distrito Otxarkoaga-Txurdinaga. Bilbao)':11756,
'Zabala (Distrito Ibaiondo. Bilbao)':14536,
'Txurdinaga (Distrito Otxarkoaga-Txurdinaga. Bilbao)':19221,
'Casco Viejo - Zazpi Kaleak (Distrito Ibaiondo. Bilbao)':19921,
'Basurtu (Distrito Basurtu-Zorrotza. Bilbao)':24509,
'Olabeaga (Distrito Basurtu-Zorrotza. Bilbao)':16783,
'San Ignazio-Elorrieta (Distrito Deustu. Bilbao)':18853,
'Bilbo Zaharra (Distrito Ibaiondo. Bilbao)':15108,
'Ibaizabal-Abusu (Distrito Ibaiondo. Bilbao)':17409,
'Ametzola (Distrito Errekalde. Bilbao)': 21511,
'Begoña (Distrito Begoña-Santutxu. Bilbao)':22492,
'Matiko-Loruri (Distrito Uribarri. Bilbao)':19129,
'Abando Ensanche (Distrito Abando. Bilbao)':35944,
'Zorrotza (Distrito Basurtu-Zorrotza. Bilbao)':15431,
}

In [371]:
renta_barrios_vall= {'Belén-Pilarica (Valladolid Capital)': 19.537,
 'Circula-Vadillos (Valladolid Capital)': 19662,
 'Las Delicias (Valladolid Capital)': 18614,
 'La Victoria (Valladolid Capital)': 24683,
 'Valladolid Capital': 'na',
 'Paseo Zorrilla-Campo Grande-Cuatro de Marzo (Valladolid Capital)': 25654,
 'Centro (Valladolid Capital)': 32596, 
 'Parquesol (Valladolid Capital)': 28580,
 'Caño Argales (Valladolid Capital)': 25912,
 'Camino de la Esperanza-Arturo León (Valladolid Capital)': 19537,
 'Barrio España-San Pedro Regalado (Valladolid Capital)': 18984,
 'Ciudad de la Comunicación-Argales (Valladolid Capital)': 17745,
 'Hospital-Rondilla-Santa Clara (Valladolid Capital)': 17745,
 'Las Villas-Covaresa-Parque Alameda-La Rubia (Valladolid Capital)': 24575,
 'Las Flores-San Isidro-Pajarillos (Valladolid Capital)': 18025,
 'Girón (Valladolid Capital)': 19537,
 'Batallas-San Juan (Valladolid Capital)': 19662, 
 'Villa de Prado (Valladolid Capital)': 25538,
 'Huerta del Rey-Arturo Eyríes (Valladolid Capital)': 24683,
 'Fuente Berrocal-La Overuela (Valladolid Capital)': 22352,
 'San Miguel-San Nicolás-San Pablo (Valladolid Capital)': 25834,
 'Universidad (Valladolid Capital)': 26676,
 'Hospital Nuevo (Valladolid Capital)': 26645,
 'Pinar de Jalón (Valladolid Capital)': 20471,
 'Polígono Industrial San Cristóbal (Valladolid Capital)':18614,
 'Pinar de Antequera-Puente Duero (Valladolid Capital)':20471   
                    }   
                    

In [374]:
renta_barrios_bilb.update(renta_barrios_vall)

In [375]:
renta_barrios=renta_barrios_bilb

In [376]:
len(renta_barrios)

65

Concateno los tres diccionarios

RENTAS_BARRIO disponible por persona por barrio

In [379]:
rentas_barrio=[]
for loc in barrios:
   
    if loc in renta_barrios.keys():  
    
        rentas_barrio.append(renta_barrios[loc])
    else:
        rentas_barrio.append('null')


In [380]:
len(rentas_barrio) #FEAT

4440

### Almaceno los datos en un csv 

In [392]:
import csv
import numpy as np

diccionario = {'titulo':titulo, 'precio': precios, 'dist_centro':dist_centro, 'long': long, 'lat':lat, 
               'long_': location_lon, 'lat_':location_lat, 'metros': metros, 'planta':planta, 'tipo_vivienda':tipo_home,
               'n_banos': baños, 'n_habitaciones': habitaciones,  'ascensor_en_desc':ascensor_desc, 
               'ascensor_en_caract':Ascensor,  'trastero_en_desc': trastero_desc, 'trastero_en_caract':Trastero,
               'garaje_en_desc':garaje_desc, 'garaje_en_caract':Garaje, 'jardin_en_desc':jardin_desc, 
               'parquet_en_desc':parquet_desc, 'obra_nueva':obra_nueva, 'barrio': barrios, 'ciudad':ciudad,
               'renta_media_barrio_persona':rentas_barrio, 'desc':descripcion}
import pandas as pd

# Creación DataFrame:
df_pisos = pd.DataFrame(diccionario)

# Guarda datos en CSV:
df_pisos.to_csv('pisos_2021.csv', header=True, index=False)



In [393]:
# Creación DataFrame para almacenar los datos de las localizaciones generadas con Geopy (elevado tiempo de compilacion):
dic_val={'lat_vall':loc_lat, 'long_vall':loc_lon,  'dist_val':dist_centro_vall}
dic_bilb={'lat_bilb':loc_lat_bilb, 'lon_bilb':loc_lon_bilb,'dist_bilb':dist_centro_bilb}
df_coord_vall=pd.DataFrame(dic_val)
df_coord_bilb=pd.DataFrame(dic_bilb)
# Guarda datos en CSV:
df_coord_bilb.to_csv('coord_bilb', header=True, index=False)
df_coord_vall.to_csv('coord_vall', header=True, index=False)